## Домашнее задание по Автоматической обработке естественного языка №1
### Настя Шабаева, БКЛ181

Оценка тональной окраски отзывов на некоторые лекарства от боли в горле с сайта https://protabletky.ru

### Импорт всего необходимого:

In [243]:
import urllib.request
import re
import os
import time
import nltk
from nltk.corpus import stopwords
stop_words = set(stopwords.words('russian'))
from pymorphy2 import MorphAnalyzer
morph = MorphAnalyzer()
from collections import Counter
from sklearn.metrics import accuracy_score

### Функция парсер

В неё поступает ссылка на страницу, с которой нужно собрать информацию. Она возвращает текст кода в кодировке utf-8.

In [244]:
def parcer(url): #функция парсер, берет ссылку, возвращает текст кода
    page = urllib.request.urlopen(url)
    text = page.read().decode('utf-8')
    return text

### Функция, которая достает со страницы текст отзывов и их рейтинг

В функцию поступает текст со страницы html. С помощью регулярных выражений производится поиск текста отзывов, рейтинга отзывов. Из текста удаляются лишние пометки и тэги.
Функция возвращает словарь, в ключах которого находятся тексты отзывов, в соответствующих значениях рейтинги данных отзывов (переменной типа string)

In [245]:
def make_clean_text_mark(raw_html): #функция, достающая текст отзывов со страницы и их рейтинг, записывает текст и рейтинг в словарь
    dict_text_mark = {}
    text_f = re.findall('<span itemprop="reviewBody"><p class="comment.*?>(.*?)</span>', raw_html, flags= re.U | re.DOTALL)
    text_mark = re.findall('Рейтинг (.*?) <span class="ser">/ 5</span>', raw_html)
    for i in range(len(text_f)):
        a = re.sub('&quot;', '', text_f[i])
        dict_text_mark[re.sub('<.*?>', '', a)] = text_mark[i+1]
    return dict_text_mark

### Функция, которая меняет тип переменной, в которой хранится рейтинг отзыва

В функцию поступает словарь (в ключах тексты отзывов, в значениях их рейтинги, в виде строк). Функция возвращает словарь, где значения переведены в тип float. 

In [246]:
def change_mark_type(dict_text_mark_func): #функция, которая в словаре делает рейтинг числом вместо строки
    for i in dict_text_mark_func.keys():
        dict_text_mark_func[i] = int(dict_text_mark_func[i][0]) + int(dict_text_mark_func[i][-1])/10
    return dict_text_mark_func

### Функция, заполняющая два списка отдельно положительными и отрицательными отзывами

Функция получает словарь (в ключах - тексты отзывов, в значениях - числовые рейтинги). Те ключи, рейтинги которых больше 2,5, добавляются в глобальный список положительных отзывов, те, рейтинг которых меньше 2,5, добавляются в глобальный список отрицательных отзывов

In [247]:
def pos_neg(clean_dict): #функция, которая делит отзывы на 2 списка по рейтингу (в одном - положительные, в другом - отрицательные)
    for i in clean_dict.keys():
        if clean_dict[i] > 2.5:
            positive.append(i)
        if clean_dict[i] < 2.5:
            negative.append(i)

### Функция, токенизирующая и лемматизирующая отзывы

В функцию поступает список отзывов. Каждый отзыв токенизируется, все буквенные токены приводятся к нижнему регистру, лемматизируются и добавляются в отдельный список. Возращает функция список лемм, встретившихся в полученном списке отзывов

In [248]:
def tokenize_lemmatize(list_comments): #функция, которая получает список комментариев и возвращает новый список, в котором хранятся все леммы, встретившиеся в комментариях из списка
    tokens = []
    for i in list_comments:
        tokens.extend(nltk.word_tokenize(i))
    tokens_lemma = []
    for i in tokens:
        if i.isalpha():
            tokens_lemma.append(morph.parse(i.lower())[0].normal_form)
    return tokens_lemma

### Функция, токенизирующая и лемматизирующая отдельный отзыв

В функцию поступает отзыв. Он токенизируется, все буквенные токены приводятся к нижнему регистру и лемматизируются, леммы добавляюся в список, который возвращает функция

In [249]:
def tokenize_lemmatize_one_comment(comment): #функция, которая получает текст комментария и возвращает список лемм, встретившихся в этом комментарии
    tokens_lemma = []
    for i in nltk.word_tokenize(comment):
        if i.isalpha():
            tokens_lemma.append(morph.parse(i.lower())[0].normal_form)
    return tokens_lemma

### Функция, которая возвращает список самых частотных "уникальных" слов

В функцию поступают 2 списка. Функция возвращает 200 самых частотных элементов первого списка, которые не являются стоп словами и не содержатся во втором списке

In [250]:
def set_making(first_list,second_list): #функция, которая получает 2 списка и возвращает 200 самых частотных элементов первого, которых нет во втором
    freqcount = Counter()
    freqlist = []
    for i in first_list:
        if (i not in second_list) and (i not in stop_words):
            freqcount[i] += 1
    for j in freqcount.most_common(200):
        freqlist.append(j[0])
    return freqlist

### Функция проверки отзывов

В функцию поступает ссылка на страницу, с которой нужно скачать отзывы для проверки, а также список частотных "уникальный" слов из положительных отзывов и такой же список для слов из отрицательных отзывов. 

Функция проходит по всем отзывам предоставленным на проверку. 
Она выясняет, сколько в каждом отзыве находится слов, содержащихся в списке слов из положительных отзывов и соответственно в списке слов из отрицательных отзывов. В зависимости от того, каких слов (положительных или отрицательных) больше в отзыве, определяется (выводится) его статус, а также каждому отзыву присваивается 0, -1 или 1 в списке check_pred (для дальнейшего вычисления accuracy).
Затем функция выводит реальный статус отзывов (отпределенный по рейтингу), а также каждому отзыву присваивается 0, -1 или 1 в списке check_true (для дальнейшего вычисления accuracy).

0 - нейтральный отзыв (одинаковое количество положительных и отрицательных слов - в списке с предположительными статусами, или же рейтинг 2,5 - в списке с реальными статусами)

-1 - отрицательный отзыв (отрицательных слов больше, чем положительных - в списке с предположительными статусами, или же рейтинг меньше 2,5 - в списке с реальными статусами)

1 - положительный отзыв (положительных слов больше, чем отрицательных - в списке с предположительными статусами, или же рейтинг больше 2,5 - в списке с реальными статусами)

In [251]:
def check(url, pos_words, neg_words):
    text_check = parcer(url)
    dict_check_clean = change_mark_type(make_clean_text_mark(text_check))
    for comment in dict_check_clean.keys():
        next_dict_check = Counter()
        for word in tokenize_lemmatize_one_comment(comment):
            if word in pos_words:
                next_dict_check['pos'] += 1
            if word in neg_words:
                next_dict_check['neg'] += 1
        print(comment, '\n')
        if next_dict_check.most_common(1)[0][0] == 'pos':
            if  next_dict_check['pos'] == next_dict_check['neg']:
                print("ДОПУСТИМ, ОТЗЫВ НЕЙТРАЛЬНЫЙ")
                check_pred.append(0)
            else:
                print("ОТЗЫВ ПОЛОЖИТЕЛЬНЫЙ")
                check_pred.append(1)
        else:
            print("ОТЗЫВ ОТРИЦАТЕЛЬНЫЙ")
            check_pred.append(-1)
        if dict_check_clean[comment] >= 2.5:
            if dict_check_clean[comment] > 2.5:
                print("ОТЗЫВ РЕАЛЬНО ПОЛОЖИТЕЛЬНЫЙ \n")
                check_true.append(1)
            else:
                print("ОТЗЫВ РЕАЛЬНО НЕЙТРАЛЬНЫЙ \n")
                check_true.append(0)
        else:
            print("ОТЗЫВ РЕАЛЬНО ОТРИЦАТЕЛЬНЫЙ \n")
            check_true.append(-1)

### Основная функция

Функция проходит по всем ссылкам, собирает информацию. Создаются списки положительных и отрицательных лемм, потом списки "уникальных" частотных слов из положительных и отрицательных отзывов. Также в функции объявляется ссылка с отзывами для проверки, вызывается функция проверки отзывов и подсчитывается (и выводится) accuracy на основе списков check_pred и check_true, заполняющихся в ходе функции check.

In [252]:
def main():
    for url in urls:
        text = parcer(url)
        time.sleep(3)
        pos_neg(change_mark_type(make_clean_text_mark(text)))
    pos_lemmas = tokenize_lemmatize(positive[:min(len(positive),len(negative))])
    neg_lemmas = tokenize_lemmatize(negative[:min(len(positive),len(negative))])
    pos_set = set_making(pos_lemmas,neg_lemmas)
    neg_set = set_making(neg_lemmas, pos_lemmas)
    url_check = 'https://protabletky.ru/kagocel/'
    check(url_check, pos_set, neg_set)
    print('Accuracy: ', accuracy_score(check_true, check_pred))

### Объявление глобальных переменных и вызов основной функции

In [253]:
urls = ['https://protabletky.ru/ingavirin/', 'https://protabletky.ru/amixin/', 'https://protabletky.ru/arbidol/', 'https://protabletky.ru/viferon/', 'https://protabletky.ru/anaferon/', 'https://protabletky.ru/rimantadine/', 'https://protabletky.ru/anaferon-for-children/', 'https://protabletky.ru/arbidol-maximum/']
negative = []
positive = []
check_pred = []
check_true = []

if __name__ == "__main__":
    main()

Ничего не понравилось, что может понравиться в фуфломицине?Все не понравилось. Абсолютная пустышка. Никакой доказательной базы. Применяется только в России, в цивилизованных странах с хорошей медициной не продается и не используется.Стыдно за коллег, которые расхваливают сие чудо. Хотя с другой стороны почему мне должно быть стыдно? Пусть стыдятся эти чудо-доктора (коллегами их не могу назвать), которые расхваливают это. Отдельный привет департаменту здравоохранения г. Москва, который включил это в перечень средств для лечения короновируса. 

ОТЗЫВ ОТРИЦАТЕЛЬНЫЙ
ОТЗЫВ РЕАЛЬНО ОТРИЦАТЕЛЬНЫЙ 

Отличный препарат. На днях очень выручил. Курс рассчитан на 4 дня. Уже на второй день приема самочувствие улучшилось, прошли основные  симптомы (сильная  головная боль и субфебрильная температура). Маленькие таблетки удобно глотать. Относительно недорогие.Очень неудобно доставать очень маленькие (диаметром около 5 мм) таблетки из блистера.Начинайте прием как можно раньше при появлении первых симпто

Достойный отечественный противовирусный препарат, применим в педиатрической практике. Необходимо применение в первые часы заболевания, в дальнейшем клинического эффекта может не наблюдаться.Не имеет доказательной базы.Данный препарат не вызывает побочных эффектов, данных о иммуностимулирующем действии препарата нет. 

ОТЗЫВ ПОЛОЖИТЕЛЬНЫЙ
ОТЗЫВ РЕАЛЬНО ПОЛОЖИТЕЛЬНЫЙ 

Кагоцел - хороший противовирусный препарат, эффективен в первые дни заболевания. При тяжелых формах ОРИ эффект слабый.Цена для данного препарата высоковата.Возможно применение как у детей, так и у взрослых. Перед применением необходима консультация специалиста. Возможны аллергические реакции по типу крапивницы, расстройства ЖКТ. 

ОТЗЫВ ПОЛОЖИТЕЛЬНЫЙ
ОТЗЫВ РЕАЛЬНО ПОЛОЖИТЕЛЬНЫЙ 

Кагоцел - отечественный фармпрепарат, относящийся к группе так называемых индукторов интерфероногенеза. Отзывы о нём самые разнообразные. От хвалебных дифирамбов до полной дискриминации как лекарственного средства. Где же кроется золотая середина?

### Что нужно улучшить в программе:

1) Возможно, если увеличить количество отзывов, по которым создаются списки "положительных" и "отрицательных" слов для проверки, точность станет более высокой. Чем больше материала, тем больше вероятность выделить те слова, которые действительно отличают положительные отзывы от отрицательных.

2) Нужно как-то разобраться с отзывы, в которых количество "положительных" и "отрицательных" слов совпадало. Непонятно, как определять тональную окраску таких отзывов, пока я называла их нейтральными, но это не очень точно. Надо что-то придумать.

3) Возможно, стоит каким-то образом попытаться отфильтровать какие-то специфические слова, например, названия препаратов. Периодически они используются очень часто, но при этом используются не во всех отзывах, поэтому просто занимают место в списке "положительных" и "отрицательных" слов. Я не уверена, но, возможно, это тоже влияет на точность.

4) Может быть, ещё было бы неплохо расширить список стоп-слов, потому что, насколько я знаю, список, который есть в nltk не очень полный.